In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://apache.mirror.digitalpacific.com.au/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-08-15 08:42:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.53MB/s    in 0.2s    

2021-08-15 08:42:36 (5.53 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema = True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [5]:
f"There are {df.count()} rows in dataset"

'There are 5331449 rows in dataset'

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
from pyspark.sql.functions import to_date

In [8]:
# CREATE TABLE review_id_table
review_id_table = df.select(['review_id', 'customer_id','product_id', 'product_parent', to_date("review_date","yyyy-mm-dd").alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R3EQSTM9PWRAL|     650634|B0091LBZSU|     578484426| 2015-01-31|
| RBWPRK17XKIXD|   19827510|B00PWW3LQ6|     456433146| 2015-01-31|
| RRSLOAF273XFC|    1520474|B00DKEWA92|     460764511| 2015-01-31|
|R3S8W9Q6SWIT8O|   23905905|B0015R3A7M|     135102038| 2015-01-31|
|R3QQ6NSLRVBFJC|   28215779|B006B8U8BG|     200666829| 2015-01-31|
|R2XYDBMHUVJCSX|   23260912|B00PFZFD8Y|     344168617| 2015-01-31|
|R1L94ESVC657A9|    7965616|B0034792JG|     291831570| 2015-01-31|
|R3DI4B8LDWFQ3K|    3443178|B000052XIA|     493678550| 2015-01-31|
|R1G7VV4HCXUQQL|   20838781|B004GCUK8A|       3902598| 2015-01-31|
|R3TTNYN1TAMUSU|   15267517|B00K7W1QAO|     633971374| 2015-01-31|
|R3AMX2D9HMWV6Z|   30105089|B0034KYA36|     854858899| 2015-01-31|
|R3PO3K321GY8HI|    1622207|B00C7UZTT8|     264608272| 2015-01

In [9]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# CREATE TABLE products (product_id + product_title)
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00N2BW2PK|Fitbit Charge HR ...|
|B001D8ZGAM|Aquasentials Easy...|
|B002NVSCQI|Beans72 Organic B...|
|B001O0BL9E|Manuka Health - M...|
|B00AYIM9Y8|BulkSupplements P...|
|B00EXPSWAI|HSI PROFESSIONAL ...|
|B00CMK8M1A|Scott Extra Soft ...|
|B00P80AV74|Monster Energy Dr...|
|B00PIOZVMA|Johnson & Johnson...|
|B00787ZQFW|MaritzMayer Raspb...|
|B0071O8B3G|Lipozene Diet Pil...|
|B005H441UE|Emphaplex 90 Caps...|
|B004WPBYUY|Vitanica Yeast Ar...|
|B00SQN2CXQ|Cloruro de Magnes...|
|B00O9NIRUY|Jimmy Orange Bran...|
|B00Y34E9ZQ|Slight Touch Cont...|
|B00ISDMQ8U|Puracy Natural La...|
|B002VWJYL6|Drive Medical Win...|
|B00NWYMDJM|Premium Prenatal ...|
|B000BABW5Q|Snorepin™ – Anti ...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# CREATE TABLE customers (customer_id + customer_count)
customer_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4919528|             5|
|   51451778|             3|
|   12713799|             1|
|    8673341|             1|
|   42146698|             2|
|    1117644|             2|
|   28058398|             1|
|   14375645|             1|
|   24540309|             1|
|   39715602|             1|
|     654272|             1|
|    9015608|             5|
|   43920023|            42|
|   38209321|             1|
|   12328685|             1|
|    2866605|             2|
|   38273165|             5|
|   41066514|             1|
|   28377689|             2|
|   17018784|             2|
+-----------+--------------+
only showing top 20 rows



In [12]:
# CREATE TABLE vine_table (review_id TEXT PRIMARY KEY,star_rating INTEGER,helpful_votes INTEGER,total_votes INTEGER,vine TEXT
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R3EQSTM9PWRAL|          3|            0|          0|   N|
| RBWPRK17XKIXD|          5|            0|          0|   N|
| RRSLOAF273XFC|          5|            1|          2|   N|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|
|R1L94ESVC657A9|          5|            0|          0|   N|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|
|R3AMX2D9HMWV6Z|          5|            0|          0|   N|
|R3PO3K321GY8HI|          1|            0|          0|   N|
| RVQ0UNYX6IEVK|          1|            0|          1|   N|
|R25E9NIGO2XQ3D|          5|            

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cbal3aw7dlkn.us-east-2.rds.amazonaws.com:5432/amazon_reviews_db"
config = {"user":"root", 
          "password": "bigdatachallenge", 
          "driver":"org.postgresql.Driver"}

In [14]:
 # Write DataFrame to the review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
# Write DataFrame to the products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write DataFrame to the customers table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to the vine table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)